# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks (1).csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 76067402,
 'calculationPrice': 'close',
 'change': 0.22,
 'changePercent': 0.00149,
 'close': 0,
 'closeSource': 'ifofical',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 154.04,
 'iexCloseTime': 1671436367535,
 'iexLastUpdated': 1709446879019,
 'iexMarketPercent': 17.4386701687678,
 'iexOpen': 150.81,
 'iexOpenTime': 1675770598023,
 'iexRealtimePrice': 156.98,
 'iexRealtimeSize': 2,
 'iexVolume': 778364,
 'lastTradeTime': 1673084065253,
 'latestPrice': 152.21,
 'latestSource': 'Close',
 'latestTime': 'October 21, 2021',
 'latestUpdate': 1702257986424,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 2515745696260,


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

30.41

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns= my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe =  final_dataframe.append(
        pd.Series(
        [
            symbol, 
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'

        ],
            index= my_columns
        ),
        ignore_index = True
)


In [8]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,162.19,51.3,N/A
1,AAL,20.71,-2.09,N/A
2,AAP,233.93,25.94,N/A
3,AAPL,149.54,29.81,N/A
4,ABBV,109.16,29.66,N/A
...,...,...,...,...
500,YUM,132.97,29.82,N/A
501,ZBH,157.24,34.89,N/A
502,ZBRA,544.27,38.13,N/A
503,ZION,65.60,6.65,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [9]:
final_dataframe.sort_values('Price-to-Earnings Ratio' , ascending = False, inplace = True)
final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace= True)
final_dataframe.drop('index', axis= 1, inplace = True)


C:\Users\Aniket Bhanderi\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,UDR,57.42,1369.61,N/A
1,FIS,129.65,852.47,N/A
2,NOW,700.55,822.01,N/A
3,DIS,175.67,283.63,N/A
4,BKNG,2524.97,249.1,N/A
5,EQIX,826.09,217.18,N/A
6,KSU,315.35,216.76,N/A
7,PAYC,560.42,189.04,N/A
8,IFF,149.32,185.53,N/A
9,HAL,25.74,162.18,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [12]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [13]:
position_size = float(portfolio_size) / len (final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])
final_dataframe

C:\Users\Aniket Bhanderi\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,UDR,57.42,1369.61,348
1,FIS,129.65,852.47,154
2,NOW,700.55,822.01,28
3,DIS,175.67,283.63,113
4,BKNG,2524.97,249.1,7
5,EQIX,826.09,217.18,24
6,KSU,315.35,216.76,63
7,PAYC,560.42,189.04,35
8,IFF,149.32,185.53,133
9,HAL,25.74,162.18,777


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [14]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol},fb&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()


# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio

pb_ratio = data['AAPL']['advanced-stats']['priceToBook']
# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']


# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterpriseValue = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterpriseValue / ebitda



# Enterprise Value divided by Gross Profit (EV/GP)
grossProfit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gp = enterpriseValue / grossProfit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [15]:
rv_columns = [
    'Ticket',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earning Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
            enterpriseValue = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            grossProfit = data[symbol]['advanced-stats']['grossProfit']

            try:
                ev_to_ebitda = enterpriseValue / ebitda
            except:
                np.NaN
            
            try:
                ev_to_gp = enterpriseValue / grossProfit
            except:
                np.NaN
                
            rv_dataframe =  rv_dataframe.append(
        
            pd.Series(
            [
            symbol, 
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gp,
            'N/A',
            'N/A',

            ],
                index= rv_columns
            ),
            ignore_index = True
            )

    

In [17]:
rv_dataframe

,Ticket,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,157.52,N/A,50.47,N/A,9.64,N/A,31.300796,N/A,14.380355,N/A,N/A
1,AAL,20.42,N/A,-2.05,N/A,-1.73,N/A,-4.527361,N/A,1.799763,N/A,N/A
2,AAP,239.29,N/A,25.5,N/A,4.62,N/A,12.926438,N/A,3.047911,N/A,N/A
3,AAPL,156.52,N/A,29.39,N/A,39.62,N/A,23.164347,N/A,18.223649,N/A,N/A
4,ABBV,110.46,N/A,30.09,N/A,15.59,N/A,10.805995,N/A,7.544512,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.54,N/A,29.24,N/A,-4.94,N/A,21.735242,N/A,10.254585,N/A,N/A
501,ZBH,152.64,N/A,35.79,N/A,2.6,N/A,21.139978,N/A,6.966199,N/A,N/A
502,ZBRA,534.34,N/A,37.99,N/A,11.17,N/A,28.595853,N/A,12.007329,N/A,N/A
503,ZION,65.83,N/A,6.45,N/A,1.41,N/A,5.132931,N/A,2.661706,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticket,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,187.30,N/A,61.13,N/A,None,N/A,18.674096,N/A,10.354418,N/A,N/A
40,AON,334.71,N/A,None,N/A,None,N/A,20.220578,N/A,1.103403,N/A,N/A
71,BRK.B,288.61,N/A,None,N/A,None,N/A,24.303405,N/A,17.745480,N/A,N/A
118,CTL,11.00,N/A,9.82,N/A,None,N/A,25.381415,N/A,13.893452,N/A,N/A
136,DISCK,25.34,N/A,None,N/A,None,N/A,6.971887,N/A,3.482842,N/A,N/A
165,ETFC,51.20,N/A,14.58,N/A,None,N/A,20.190193,N/A,18.704120,N/A,N/A
186,FLIR,59.77,N/A,33.38,N/A,None,N/A,9.057082,N/A,3.608753,N/A,N/A
190,FOX,41.42,N/A,None,N/A,None,N/A,12.785902,N/A,7.459612,N/A,N/A
204,GOOG,2905.17,N/A,None,N/A,None,N/A,2.779899,N/A,2.423664,N/A,N/A
325,MXIM,106.32,N/A,34.29,N/A,None,N/A,5.722494,N/A,7.224821,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [19]:
for column in ['Price-to-Earning Ratio','Price-to-Book Ratio','EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [20]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticket,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [25]:
metrics = {
    'Price-to-Earning Ratio' : 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile' }

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row,metrics[metric]] = stats.percentileofscore(rv_dataframe[metric],rv_dataframe.loc[row,metric])


In [32]:
rv_dataframe

,Ticket,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,157.52,N/A,50.47,83.5644,9.64,79.2079,31.300796,86.9307,14.380355,80.7921,N/A
1,AAL,20.42,N/A,-2.05,7.52475,-1.73,4.9505,-4.527361,2.57426,1.799763,7.92079,N/A
2,AAP,239.29,N/A,25.50,48.7129,4.62,56.2376,12.926438,40,3.047911,14.4554,N/A
3,AAPL,156.52,N/A,29.39,57.4257,39.62,96.0396,23.164347,72.0792,18.223649,88.9109,N/A
4,ABBV,110.46,N/A,30.09,59.4059,15.59,88.5149,10.805995,30.099,7.544512,49.703,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.54,N/A,29.24,57.2277,-4.94,4.55446,21.735242,70.297,10.254585,65.3465,N/A
501,ZBH,152.64,N/A,35.79,68.9109,2.60,33.7624,21.139978,69.901,6.966199,44.5545,N/A
502,ZBRA,534.34,N/A,37.99,71.9802,11.17,81.9802,28.595853,84.3564,12.007329,73.8614,N/A
503,ZION,65.83,N/A,6.45,8.81188,1.41,12.8713,5.132931,5.74257,2.661706,12.8713,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [34]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row,'RV Score'] = mean(value_percentiles)

In [35]:
rv_dataframe

,Ticket,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,157.52,N/A,50.47,83.5644,9.64,79.2079,31.300796,86.9307,14.380355,80.7921,82.6238
1,AAL,20.42,N/A,-2.05,7.52475,-1.73,4.9505,-4.527361,2.57426,1.799763,7.92079,5.74257
2,AAP,239.29,N/A,25.50,48.7129,4.62,56.2376,12.926438,40,3.047911,14.4554,39.8515
3,AAPL,156.52,N/A,29.39,57.4257,39.62,96.0396,23.164347,72.0792,18.223649,88.9109,78.6139
4,ABBV,110.46,N/A,30.09,59.4059,15.59,88.5149,10.805995,30.099,7.544512,49.703,56.9307
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.54,N/A,29.24,57.2277,-4.94,4.55446,21.735242,70.297,10.254585,65.3465,49.3564
501,ZBH,152.64,N/A,35.79,68.9109,2.60,33.7624,21.139978,69.901,6.966199,44.5545,54.2822
502,ZBRA,534.34,N/A,37.99,71.9802,11.17,81.9802,28.595853,84.3564,12.007329,73.8614,78.0446
503,ZION,65.83,N/A,6.45,8.81188,1.41,12.8713,5.132931,5.74257,2.661706,12.8713,10.0743


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [41]:
rv_dataframe.sort_values('RV Score',  ascending = True,  inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticket,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,BA,221.81,N/A,-14.59,5.14851,-7.6100,4.35644,-30.743391,0.792079,-212.412710,0.39604,2.67327
1,PRU,113.61,N/A,6.45,8.81188,0.7011,5.54455,5.035505,5.34653,0.682014,1.38614,5.27228
2,UNM,28.70,N/A,8.22,11.6832,0.5226,5.14851,3.609825,3.9604,0.418185,0.990099,5.44554
3,AAL,20.42,N/A,-2.05,7.52475,-1.7300,4.9505,-4.527361,2.57426,1.799763,7.92079,5.74257
4,NCLH,26.50,N/A,-2.30,7.32673,1.5200,15.3465,-9.683569,2.17822,-78.693352,0.594059,6.36139
5,C,70.59,N/A,7.47,10.495,0.8030,6.53465,4.501760,4.55446,1.703457,7.32673,7.22772
6,L,57.80,N/A,10.51,15.6436,0.8491,6.93069,5.552859,6.53465,0.969941,2.57426,7.92079
7,ALL,127.01,N/A,10.41,15.2475,1.4800,14.2574,2.418070,3.36634,0.791481,1.78218,8.66337
8,COF,168.20,N/A,6.62,9.10891,1.2000,10.198,4.968031,4.75248,2.267608,10.8911,8.73762
9,KSS,49.40,N/A,10.08,14.6535,1.3700,12.4752,4.045509,4.15842,1.146903,3.76238,8.76238


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [42]:
portfolio_input()

Enter the value of your portfolio:1000000


In [94]:
position_size = float(portfolio_size) / len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row,['Number of Shares to Buy']] = math.floor(position_size / rv_dataframe.loc[row,'Price'])
rv_dataframe

,Ticket,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,BA,221.81,90,-14.59,5.14851,-7.6100,4.35644,-30.743391,0.792079,-212.412710,0.39604,2.67327
1,PRU,113.61,176,6.45,8.81188,0.7011,5.54455,5.035505,5.34653,0.682014,1.38614,5.27228
2,UNM,28.70,696,8.22,11.6832,0.5226,5.14851,3.609825,3.9604,0.418185,0.990099,5.44554
3,AAL,20.42,979,-2.05,7.52475,-1.7300,4.9505,-4.527361,2.57426,1.799763,7.92079,5.74257
4,NCLH,26.50,754,-2.30,7.32673,1.5200,15.3465,-9.683569,2.17822,-78.693352,0.594059,6.36139
5,C,70.59,283,7.47,10.495,0.8030,6.53465,4.501760,4.55446,1.703457,7.32673,7.22772
6,L,57.80,346,10.51,15.6436,0.8491,6.93069,5.552859,6.53465,0.969941,2.57426,7.92079
7,ALL,127.01,157,10.41,15.2475,1.4800,14.2574,2.418070,3.36634,0.791481,1.78218,8.66337
8,COF,168.20,118,6.62,9.10891,1.2000,10.198,4.968031,4.75248,2.267608,10.8911,8.73762
9,KSS,49.40,404,10.08,14.6535,1.3700,12.4752,4.045509,4.15842,1.146903,3.76238,8.76238


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [95]:
writer = pd.ExcelWriter('Value_Stragey.xlsx' , engine='xlsxwriter')
rv_dataframe.to_excel(writer,sheet_name = 'Value Strategy' , index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [96]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [97]:
column_formats = {
    'A' : ['Ticker', string_template],
    'B' : ['Price', dollar_template],
    'C' : ['Number of Shares to Buy', integer_template],
    'D' : ['Price-to-Earning Ratio',float_template],
    'E' : ['PE Percentile', percent_template],
    'F' : ['Price-to-Book Ratio', float_template],
    'G':  ['PB Percentile', percent_template],
    'H' : ['EV/EBITDA', float_template],
    'I' : ['EV/EBITDA Percentile', percent_template],
    'J' : ['EV/GP', float_template],
    'K' : ['EV/GP Percentile' , percent_template],
    'L' : ['RV Score', float_template]
}


for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}' , 22 , column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [98]:
writer.save()